In [41]:
import numpy as np
import pygpufit.gpufit as gf
import matplotlib.pyplot as plt

In [42]:
def generate_gauss_1d(parameters, x):
    """
    Generates a 1D Gaussian curve.

    :param parameters: The parameters (a, x0, s, b)
    :param x: The x values
    :return: A 1D Gaussian curve.
    """

    a = parameters[0]
    x0 = parameters[1]
    s = parameters[2]
    b = parameters[3]

    y = a * np.exp(-np.square(x - x0) / (2 * s**2)) + b

    return y

In [43]:

# constants
n_fits = 2
n_points = 40
n_parameter = 4  # model will be GAUSS_1D

# true parameters
true_parameters = np.array((40, 20, 5, 10), dtype=np.float32)

# generate data
data = np.empty((n_fits, n_points), dtype=np.float32)

x = np.arange(n_points, dtype=np.float32)
data[0, :] = generate_gauss_1d(true_parameters, x)
data[1, :] = generate_gauss_1d(true_parameters - 2, x)

noise = np.random.uniform(low=-1, high=1, size=(n_fits, n_points))

data += noise

# tolerance
tolerance = 0.001

# max_n_iterations1
max_n_iterations = 100

# model id
model_id = gf.ModelID.GAUSS_1D

# initial parameters
initial_parameters = np.empty((n_fits, n_parameter), dtype=np.float32)
initial_parameters[0, :] = (30, 15, 30, 7)

# # call to gpufit
# parameters, states, chi_squares, number_iterations, execution_time = gf.fit(data, None, model_id,
#                                                                             initial_parameters, tolerance, \
#                                                                             max_n_iterations, None, None, None)

gf.ConstraintType.LOWER_UPPER
constraint_types = np.empty((n_parameter), dtype=np.int32)
constraint_types[:] = gf.ConstraintType.LOWER_UPPER

print(f"len(constraint_types): {len(constraint_types)}")

constraints = np.empty((n_fits, n_parameter * 2), dtype=np.float32)

c1 = (30, 50, 10, 30, 2, 7, 9, 11)
c2 = tuple([x - 3 for x in c1])

constraints[0, :] = np.array(c1, dtype=np.float32)
constraints[1, :] = np.array(c2, dtype=np.float32)

#call to gpufit
parameters, states, chi_squares, number_iterations, execution_time = gf.fit_constrained(data, None, model_id,
                                                                            initial_parameters, constraints, constraint_types, tolerance, \
                                                                            max_n_iterations, None, None, None)


len(constraint_types): 4


RuntimeError: status = -1, message = b'invalid argument'

In [ ]:
data_measured1 = data[0]
data_fit1 = generate_gauss_1d(parameters[0], x)

data_measured2 = data[1]
data_fit2 = generate_gauss_1d(parameters[1], x)


plt.plot(x, data_measured1, linestyle="dotted", label="data")
plt.plot(x, data_fit1, linestyle="dashed", label="fit")
plt.plot(x, data_measured2, linestyle="dotted", label="data")
plt.plot(x, data_fit2, linestyle="dashed", label="fit")


NameError: name 'parameters' is not defined

In [ ]:
for i in range(n_fits):
    # print(f"len(constraints[i]) % 2: {len(constraints[i]) % 2}")
    print("constraints: ", end="")
    for x in range(len(constraints[i]) // 2):
        print(f"{constraints[i][x*2]:.1f}"+" - "+f"{constraints[i][x*2+1]:.1f}   ", end="")
    print("")
    print("parameters:     ", end="")
    for y in parameters[i]:
        print(f"{y:.1f}"+"          ", end="")
    print("")
    print("")

constraints: 30.0 - 50.0   10.0 - 30.0   2.0 - 7.0   9.0 - 11.0   
parameters:     39.7          20.0          5.0          9.9          

constraints: 27.0 - 47.0   7.0 - 27.0   -1.0 - 4.0   6.0 - 8.0   
parameters:     0.0          0.0          0.0          0.0          

